<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/06b_GPT2_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import libraries, make sure you don't reinstall gpt_2_simple if you have already sone so by commenting that line

In [2]:
%tensorflow_version 1.x
#!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Load and check the data

In [3]:
speeches = pd.read_pickle("all_speech_sentence_filtered.pkl")

print("Number of speeches: {}".format(len(speeches)))
speeches = speeches.head(5000)

Number of speeches: 298792


Make .txt files for republican and democrat data as gpt_2_simple needs the data in this format

In [4]:
# make separate txt files for republicans and democrats
with open("rep_speeches.txt", 'w') as f1, open("dem_speeches.txt", 'w') as f2:
  for i in range(len(speeches)):
    speech = speeches.iloc[i].Questions
    party = speeches.iloc[i].Party
    if party == 'R':
      f1.write(speech + '\n')
    else:
      f2.write(speech + '\n')

# check files 
with open('rep_speeches.txt', 'r') as f:
  lines = f.readlines()
  for line in lines[:5]:
    print(line)

as all Members are aware, I have been working for some time on various rules changes that would ensure a more efficient process by which the Senate considers appropriations bills. One of our concerns has been reinstating rule XVI with respect to legislation on appropriations bills, I believe that many of the extraneous items that have been added to appropriations bills over the past few Congresses would have been ruled out of order if the Senate still had the ability to raise a point of order against legislation on appropriations bills formerly contained in rule XVI. Other rule abuses occurred during the closing days of the 105th Congress. Consequently, I will shortly introduce five Senate resolutions regarding rules and budget process changes and will ask for their proper referral. Once the resolutions have been referred to the appropriate committee, it is my hope the chairmen of the committees will begin swift committee work on the resolutions so the committees can act on the changes

Finetunine GPT-2

In [7]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 224Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 73.6Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 230Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 169Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 320Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 139Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 129Mit/s]                                                       


In [5]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pwd

/content/drive/My Drive


Train for republican data

In [7]:
file_name = "rep_speeches.txt"
gpt2.copy_file_from_gdrive(file_name)

In [10]:
RUN_NAME = 'rep'
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=200,
              restore_from='fresh',
              run_name=RUN_NAME,
              print_every=10,
              sample_every=200,
              save_every=500
              )

gpt2.copy_checkpoint_to_gdrive(run_name=RUN_NAME)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:34<00:00, 94.90s/it]


dataset has 16267829 tokens
Training...
[10 | 53.45] loss=3.51 avg=3.51
[20 | 99.52] loss=3.14 avg=3.33
[30 | 145.48] loss=3.12 avg=3.26
[40 | 191.37] loss=3.22 avg=3.25
[50 | 237.30] loss=3.18 avg=3.23
[60 | 283.19] loss=3.05 avg=3.20
[70 | 329.14] loss=3.05 avg=3.18
[80 | 375.09] loss=2.91 avg=3.14
[90 | 421.04] loss=3.02 avg=3.13
[100 | 466.95] loss=3.18 avg=3.14
[110 | 513.01] loss=3.11 avg=3.13
[120 | 559.01] loss=3.03 avg=3.12
[130 | 605.08] loss=3.12 avg=3.12
[140 | 651.16] loss=3.14 avg=3.12
[150 | 697.18] loss=2.99 avg=3.11
[160 | 743.29] loss=3.12 avg=3.11
[170 | 789.48] loss=3.03 avg=3.11
[180 | 835.63] loss=2.96 avg=3.10
[190 | 881.76] loss=2.94 avg=3.09
[200 | 927.88] loss=2.98 avg=3.09
Saving checkpoint/run1/model-200


Make sure you restart runtime before running this cell!

In [3]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=RUN_NAME)

Loading checkpoint checkpoint/run1/model-200
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-200


Generate republican answers

In [7]:
questions = ["Should abortion be illegal?",
             "What do you believe about tax increases?",
             "Should same-sex marriage be legal?",
             "Is climate change real?",
             "Should immigrants be allowed to obtain citizenship?",
             "Who should be given voting rights?",
             "Should we have higher taxes for higher incomes?",
             "Should we allow death penalty?",
             "Should we have universal healthcare?",
             "Do government regulations hinder free market capitalism?",
             "What do you think about the current president?",
             "Should we reduce national debt?",
             "Should we increase spending on healthcare?",
             "Should we increase spending on education?",
             "Should every American have equal opportunities regardless of sex, age and race?",
             "Should be introduce more gun control measures?",
             "What do you think about the current president?",
             "Should Americans be free?",
             "What party do you support?",
             "What is the biggest threat to America?"
             ]

In [9]:
with open("rep_generated.txt", 'w') as f:
  for i in range(len(questions)):
    text = gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix=questions[i],
              nsamples=1,
              batch_size=1,
              return_as_list=True)[0]
    print(text)
    f.write(text + '\n')

Should abortion be illegal? We do not want to say that. We do not want to say that our laws are wrong. So I am open to the idea of legislation to ease the issue. No. We will have to make sure that we do not do that. But I will say something that is obviously wrong. We have laws that are just too complicated, too complicated, and it is difficult to do the same thing on a State level. We put in this system where we have a number of different rules that can be applied. What I am trying to do is to get to the bottom of many of these problems. What I am trying to do is to find some practical solutions. We have laws that we can pass in the State level that will keep it from being overruled by the Supreme Court. We will ban abortions under federal law, but we will not allow abortion rights to be dictated to the Federal Government. So we will have to find ways to deal with these laws so that they are not so complicated. We will have to find ways to deal with the many problems that we have when

Train for democrat data

In [ ]:
file_name = "dem_speeches.txt"
gpt2.copy_file_from_gdrive(file_name)

sess = gpt2.start_tf_sess()
RUN_NAME = 'dem'

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=200,
              restore_from='fresh',
              run_name=RUN_NAME,
              print_every=10,
              sample_every=200,
              save_every=500
              )

gpt2.copy_checkpoint_to_gdrive(run_name=RUN_NAME)

Make sure you restart runtime before running this cell!

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=RUN_NAME)

Generate republican answers

In [ ]:
questions = ["Should abortion be illegal?",
             "What do you believe about tax increases?",
             "Should same-sex marriage be legal?",
             "Is climate change real?",
             "Should immigrants be allowed to obtain citizenship?",
             "Who should be given voting rights?",
             "Should we have higher taxes for higher incomes?",
             "Should we allow death penalty?",
             "Should we have universal healthcare?",
             "Do government regulations hinder free market capitalism?",
             "What do you think about the current president?",
             "Should we reduce national debt?",
             "Should we increase spending on healthcare?",
             "Should we increase spending on education?",
             "Should every American have equal opportunities regardless of sex, age and race?",
             "Should be introduce more gun control measures?",
             "What do you think about the current president?",
             "Should Americans be free?",
             "What party do you support?",
             "What is the biggest threat to America?"
             ]

In [ ]:
with open("dem_generated.txt", 'w') as f:
  for i in range(len(questions)):
    text = gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix=questions[i],
              nsamples=1,
              batch_size=1,
              return_as_list=True)[0]
    print(text)
    f.write(text + '\n')